Here we test how mesh granularity and element degree affect error of FEM.

In [ ]:
import os
import random
import itertools

import numpy as np
import pandas as pd

import FEM.fem_slice_point_new as fspn
import FEM.fem_common as fc

# Preprocessing

We choose $N$ random points in the region of interest (here cube with edges of 0.3 mm length).

In [ ]:
N = 100

In [ ]:
H = 3e-4
GEOMETRY = 'circular_slice'
PROPERTIES = 'FEM/model_properties/circular_slice.ini'

In [ ]:
MESHES = ['coarsest',
          'coarser',
          'coarse',
          'normal',
          'fine',
          'finer',
          'finest',
          ]

DEGREES = [1, 2, 3]

CONFIGS = list(itertools.product(MESHES, DEGREES))

In [ ]:
MESH_DIR = 'FEM/meshes/meshes/'

In [ ]:
RESULT_DIR = os.path.join('test_FEM_parameters',
                          GEOMETRY)
os.makedirs(RESULT_DIR, exist_ok=True)

In [ ]:
POINTS_FILE = os.path.join(RESULT_DIR,
                           'point.csv')

if os.path.exists(POINTS_FILE):
    print(POINTS_FILE, 'found')
    POINTS = pd.read_csv(POINTS_FILE,
                         index_col=0)

else:
    random.seed(42)
    POINTS = np.full((N, 3), np.nan)

    for i in range(N):
        x = random.uniform(-H/2, H/2)
        y = random.uniform(-H/2, H/2)
        z = random.uniform(0, H)
        POINTS[i, :] = x, y, z

    POINTS = pd.DataFrame(POINTS, columns=['X', 'Y', 'Z'])
    POINTS.to_csv(POINTS_FILE,
                  index_label='ID')

The points are saved so any further calculations may reuse them.

# FEM benchmarking

For every granularity and degree configuration, for every of $N$ points, we use FEM to calculate appropriate correction for kCSD-like approximation of 1A point source.  The correction is probed at $N$ points and the $N\times{}N$ values are saved for further analysis.

In [ ]:
setup_time = fc.fc.Stopwatch()
total_solving_time = fc.fc.Stopwatch()

In [ ]:
for mesh, degree in CONFIGS:
    print(mesh, degree)
    result_file = os.path.join(RESULT_DIR,
                               f'{mesh}_{degree}.csv')
    
    if os.path.exists(result_file):
        print(' ', result_file, 'found')
        continue
    
    DF = []
    with setup_time:
        function_manager = fc.FunctionManager(os.path.join(MESH_DIR,
                                                           GEOMETRY,
                                                           f'{mesh}.xdmf'),
                                              degree,
                                              'CG')
        fem = fspn.SlicePointSourcePotentialFEM(function_manager,
                                                PROPERTIES)

    for src, SRC in POINTS.iterrows():
        print(mesh, degree, src)
        with total_solving_time:
            potential_corr = fem.correction_potential(SRC.X, SRC.Y, SRC.Z)
            
        for dst, DST in POINTS.iterrows():
            DF.append({
                'SRC': src,
                'DST': dst,
                'CORR': potential_corr(DST.X, DST.Y, DST.Z),
                'SOLVING_TIME': float(total_solving_time),
                'SETUP_TIME': float(setup_time)
            })
            
    DF = pd.DataFrame(DF)
    DF.to_csv(result_file, index=False)

# Analysis

In [ ]:
import matplotlib.pyplot as plt

from local import cbf

In [ ]:
import configparser
import operator
from kesi import common

In [ ]:
config = configparser.ConfigParser()
config.read(PROPERTIES)

In [ ]:
SLICE_CONDUCTIVITY = config.getfloat('slice', 'conductivity')
SALINE_CONDUCTIVITY = config.getfloat('saline', 'conductivity')
GLASS_CONDUCTIVITY = 0.0

MOI_N = 256

## Ground truth correction

As the circular slice geometry may be approximated by an "infinite slice" geometry, it is prossible to calculate true values of correction with method of images (MOI).

In [ ]:
WTG = float(SLICE_CONDUCTIVITY - GLASS_CONDUCTIVITY) / (SLICE_CONDUCTIVITY + GLASS_CONDUCTIVITY)
WTS = float(SLICE_CONDUCTIVITY - SALINE_CONDUCTIVITY) / (SLICE_CONDUCTIVITY + SALINE_CONDUCTIVITY)

GT_CORRECTION = np.full((N, N), np.nan)

for src, SRC in POINTS.iterrows():
    print(src)
    weights = []
    sources = []
    for i in range(MOI_N):
        weights.append(WTG**i * WTS**(i+1))
        sources.append(common.PointSource(SRC.X,
                                          SRC.Y,
                                          2 * (i + 1) * H - SRC.Z,
                                          conductivity=SLICE_CONDUCTIVITY))
        weights.append(WTG**(i+1) * WTS**i)
        sources.append(common.PointSource(SRC.X,
                                          SRC.Y,
                                          -2 * i * H - SRC.Z,
                                          conductivity=SLICE_CONDUCTIVITY))

    for i in range(1, MOI_N + 1):
        weights.append((WTG * WTS)**i)
        sources.append(common.PointSource(SRC.X,
                                          SRC.Y,
                                          SRC.Z + 2 * i * H,
                                          conductivity=SLICE_CONDUCTIVITY))
        weights.append((WTG * WTS)**i)
        sources.append(common.PointSource(SRC.X,
                                          SRC.Y,
                                          SRC.Z - 2 * i * H,
                                          conductivity=SLICE_CONDUCTIVITY))

#     positive = [(w, s) for w, s in zip(weights, sources) if w > 0]
#     positive.sort(key=operator.itemgetter(0), reverse=False)
#     negative = [(w, s) for w, s in zip(weights, sources) if w < 0]
#     negative.sort(key=operator.itemgetter(0), reverse=True)

    for dst, DST in POINTS.iterrows():
        values = [w * s.potential(DST.X, DST.Y, DST.Z)
                  for w, s in zip(weights, sources)]
        positive = sorted([v for v in values if v > 0])
        negative = sorted([-v for v in values if v < 0])
        GT_CORRECTION[src, dst] = sum(positive) - sum(negative)

To validate MOI we calculate the maximal reciprocity error
(due to reciprocity the ideal `GT_CORRECTION` matrix is
symmetrical).  We also estimate order of magnitude of the
off-diagonal elements.

> The diagonal elements are insignificant, as the base potential
> ($V_{base} \propto \frac{1}{r}$) is a singularity for $r = 0$.
> Moreover, for points near to the medium interface the amplitude
> of the correction potential is high at the point source location
> due to the proximity of its (source's) image reflected by
> the interface.

In [ ]:
OFF_DIAGONAL_IDX = ~np.eye(N, dtype=bool)

print('Maximal reciprocity error:', abs(GT_CORRECTION - GT_CORRECTION.T).max())
_OFF_DIAGONAL = GT_CORRECTION[OFF_DIAGONAL_IDX]
print('Linf:', abs(_OFF_DIAGONAL).max())
print('L2:', np.sqrt(np.square(_OFF_DIAGONAL).mean()))
print('L1:', abs(_OFF_DIAGONAL).mean())
print('Median absolute value:', np.median(abs(_OFF_DIAGONAL)))
print('min, med, max:', _OFF_DIAGONAL.min(), np.median(_OFF_DIAGONAL), _OFF_DIAGONAL.max())

## Base potential

To enable estimation of the relative error of potential (leadfield due to reciprocity) rather than correction (which may inflate the error where correction is small), we calculate the `BASE_POTENTIAL` matrix and then the `GT_POTENTIAL`.

In [ ]:
BASE_POTENTIAL = np.full((N, N), np.nan)

for src, SRC in POINTS.iterrows():
    _src = common.PointSource(SRC.X,
                              SRC.Y,
                              SRC.Z,
                              conductivity=SLICE_CONDUCTIVITY)
    for dst, DST in POINTS.iterrows():
        BASE_POTENTIAL[src, dst] = _src.potential(DST.X, DST.Y, DST.Z)

In [ ]:
print('Maximal reciprocity error:', abs(BASE_POTENTIAL - BASE_POTENTIAL.T)[OFF_DIAGONAL_IDX].max())
_OFF_DIAGONAL = BASE_POTENTIAL[OFF_DIAGONAL_IDX]
print('Linf:', abs(_OFF_DIAGONAL).max())
print('L2:', np.sqrt(np.square(_OFF_DIAGONAL).mean()))
print('L1:', abs(_OFF_DIAGONAL).mean())
print('Median absolute value:', np.median(abs(_OFF_DIAGONAL)))
print('min, med, max:', _OFF_DIAGONAL.min(), np.median(_OFF_DIAGONAL), _OFF_DIAGONAL.max())

In [ ]:
GT_POTENTIAL = BASE_POTENTIAL + GT_CORRECTION

In [ ]:
print('Maximal reciprocity error:', abs(GT_POTENTIAL - GT_POTENTIAL.T)[OFF_DIAGONAL_IDX].max())
_OFF_DIAGONAL = GT_POTENTIAL[OFF_DIAGONAL_IDX]
print('Linf:', abs(_OFF_DIAGONAL).max())
print('L2:', np.sqrt(np.square(_OFF_DIAGONAL).mean()))
print('L1:', abs(_OFF_DIAGONAL).mean())
print('Median absolute value:', np.median(abs(_OFF_DIAGONAL)))
print('min, med, max:', _OFF_DIAGONAL.min(), np.median(_OFF_DIAGONAL), _OFF_DIAGONAL.max())

## Reading correction matrices

We read the saved correction values

In [ ]:
labels = []
corrections = []

for mesh, degree in CONFIGS:
    print(mesh, degree)
    result_file = os.path.join(RESULT_DIR,
                               f'{mesh}_{degree}.csv')
    if not os.path.exists(result_file):
        print(' not found, skipping')
        continue

    labels.append(f'{mesh} {degree}')

    DF = pd.read_csv(result_file)
    CORR = np.full((N, N), np.nan)
    
    for row in DF.itertuples():
        CORR[row.SRC, row.DST] = row.CORR
        
    corrections.append(CORR)

## Reciprocity validation

The first test is a simple verification, whether the reciprocity is held by the result.
Reciprocity is crucial, as whole kESI optimization relies on it.

In [ ]:
reciprocity_errors = [A - A.T for A in corrections]

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Reciprocity errors [V]')
plt.yscale('symlog')
_ = plt.boxplot([np.ravel(A) for A in reciprocity_errors],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Module of reciprocity errors [V]')
plt.yscale('log')
plt.grid()
_ = plt.violinplot([A[A > 0] for A in reciprocity_errors])
_ = plt.boxplot([A[A > 0] for A in reciprocity_errors],
                labels=labels)
# _ = plt.violinplot([A[A > 0] for A in reciprocity_errors])
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Module of reciprocity errors [%]')
plt.yscale('log')
plt.grid()
_ = plt.violinplot([100 * (A / GT_POTENTIAL)[A > 0] for A in reciprocity_errors])
_ = plt.boxplot([100 * (A / GT_POTENTIAL)[A > 0] for A in reciprocity_errors],
                labels=labels)
# _ = plt.violinplot([A[A > 0] for A in reciprocity_errors])
_ = plt.xticks(rotation=30)

## Solution errors

In [ ]:
gt_errors = [A - GT_CORRECTION for A in corrections]

In [ ]:
gt_relative_errors = [(A / GT_POTENTIAL)[OFF_DIAGONAL_IDX] for A in gt_errors]

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Errors [V]')
plt.yscale('symlog')
_ = plt.boxplot([np.ravel(A) for A in gt_errors],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Module of errors [V]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([abs(np.ravel(A)) for A in gt_errors],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Errors (diagonal excluded) [V]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([abs(A[OFF_DIAGONAL_IDX]) for A in gt_errors],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Modules of errors [%]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([100 * abs(A) for A in gt_relative_errors],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
true_error_L1 = np.array([abs(_DIFF[OFF_DIAGONAL_IDX]).mean() for _DIFF in gt_errors])
true_error_L2 = np.array([np.sqrt(np.square(_DIFF[OFF_DIAGONAL_IDX]).mean()) for _DIFF in gt_errors])
true_error_Linf = np.array([abs(_DIFF[OFF_DIAGONAL_IDX]).max() for _DIFF in gt_errors])
true_error_bias = np.array([_DIFF[OFF_DIAGONAL_IDX].mean() for _DIFF in gt_errors])

In [ ]:
true_relative_error_L1 = np.array([abs(_DIFF).mean() for _DIFF in gt_relative_errors])
true_relative_error_L2 = np.array([np.sqrt(np.square(_DIFF).mean()) for _DIFF in gt_relative_errors])
true_relative_error_Linf = np.array([abs(_DIFF).max() for _DIFF in gt_relative_errors])
true_relative_error_bias = np.array([_DIFF.mean() for _DIFF in gt_relative_errors])

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Errors (diagonal excluded) [V]')

plt.plot(true_error_L1, label='L1', marker='o')
plt.plot(true_error_L2, label='L2', marker='+')
plt.plot(true_error_Linf, label='L\u221e')
plt.plot(true_error_bias, label='bias')
plt.yscale('symlog', linthresh=0.1)
plt.xticks(range(len(labels)), labels)
plt.grid()
plt.legend(loc='best')
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Errors [%]')

plt.plot(true_relative_error_L1 * 100, label='L1', marker='o')
plt.plot(true_relative_error_L2 * 100, label='L2', marker='+')
plt.plot(true_relative_error_Linf * 100, label='L\u221e')
plt.plot(true_relative_error_bias * 100, label='bias')
plt.yscale('symlog', linthresh=0.1)
plt.xticks(range(len(labels)), labels)
plt.grid()
plt.legend(loc='best')
_ = plt.xticks(rotation=30)

## Convergence validation

As for spherical geometries no exact values of correction are known,
we have to approximate the exact correction as an average of its approximations.

In [ ]:
# We approximate exact solution as average of the most advanced FEM configurations
# (in terms of mesh density and element degree)

_corrections = dict(zip(labels, corrections))

AVG = 0.5 * (_corrections['finer 3'] + _corrections['finest 2'])

In [ ]:
# # The algorithm below approximates the exact correction
# # (`_AVG`) as an average of a set of its numeric approximations.
# # Aproximation errors are then estimated with `_AVG`
# # and the worst numeric approximation is excluded from the set.
# # The `_AVG` is reevaluated and the whole process is repeated
# # as long as only 2 approximations are left in the set

# _corrections = dict(zip(labels, corrections))

# removed = []
# removal_score = []
# scores = []

# _score = 1
# while len(_corrections) > 2 and _score:
#     _AVG = sum(_corrections.values()) / len(_corrections)
#     _EST_POT = BASE_POTENTIAL + _AVG
#     _score = 0
#     scores.append([])
#     for _k, _CORR in _corrections.items():
# #         _s = np.sqrt(np.square(((_AVG - _CORR))[OFF_DIAGONAL_IDX]).mean())
#         _s = np.sqrt(np.square(((_AVG - _CORR) / _EST_POT)[OFF_DIAGONAL_IDX]).mean())
#         scores[-1].append(_s)
#         if _s > _score:
#             _score = _s
#             _key = _k
            
#     removed.append(_key)
#     removal_score.append(_score)
#     del _corrections[_key]

# AVG = sum(_corrections.values()) / len(_corrections)

In [ ]:
# _EST_POT = BASE_POTENTIAL + AVG

# for k, v in zip(removed, removal_score):
#     print(f'  {k}\t{v}')
    
# for k, v in zip(labels, corrections):
#     if k in removed:
#         continue
    
#     minimal_score = np.sqrt(np.square(((v - AVG) / _EST_POT)[OFF_DIAGONAL_IDX]).mean())
#     print(f'> {k}\t{minimal_score}')

In [ ]:
# plt.plot(removal_score)
# plt.axhline(minimal_score)
# plt.yscale('log')
# plt.grid()

In [ ]:
# for i, _scores in enumerate(scores):
#     plt.scatter([i] * len(_scores), _scores)
# plt.yscale('linear')

In [ ]:
# for i, _scores in enumerate(scores):
#     plt.scatter([i] * len(_scores), _scores)
# plt.yscale('log')

In [ ]:
diffs = [_CORR - AVG for _CORR in corrections]
error_L1 = np.array([abs(_DIFF[OFF_DIAGONAL_IDX]).mean() for _DIFF in diffs])
error_L2 = np.array([np.sqrt(np.square(_DIFF[OFF_DIAGONAL_IDX]).mean()) for _DIFF in diffs])
error_Linf = np.array([abs(_DIFF[OFF_DIAGONAL_IDX]).max() for _DIFF in diffs])
error_bias = np.array([_DIFF[OFF_DIAGONAL_IDX].mean() for _DIFF in diffs])

In [ ]:
diffs_relative = [_DIFF / (AVG + BASE_POTENTIAL) for _DIFF in diffs]
error_relative_L1 = np.array([abs(_DIFF[OFF_DIAGONAL_IDX]).mean() for _DIFF in diffs_relative])
error_relative_L2 = np.array([np.sqrt(np.square(_DIFF[OFF_DIAGONAL_IDX]).mean()) for _DIFF in diffs_relative])
error_relative_Linf = np.array([abs(_DIFF[OFF_DIAGONAL_IDX]).max() for _DIFF in diffs_relative])
error_relative_bias = np.array([_DIFF[OFF_DIAGONAL_IDX].mean() for _DIFF in diffs_relative])

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Modulus of convergence errors [V]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([abs(np.ravel(A)) for A in diffs],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Modulus of convergence errors (diagonal excluded) [V]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([abs(A[OFF_DIAGONAL_IDX]) for A in diffs],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Modulus of convergence errors (diagonal included) [%]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([100 * abs(np.ravel(A)) for A in diffs_relative],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Modulus of convergence errors (diagonal excluded) [%]')
plt.yscale('log')
plt.grid()
_ = plt.boxplot([100 * abs(A[OFF_DIAGONAL_IDX]) for A in diffs_relative],
                labels=labels)
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Convergence errors [V]')

plt.plot(error_L1, label='L1')
plt.plot(error_L2, label='L2')
plt.plot(error_Linf, label='L\u221e')
plt.plot(error_bias, label='bias')
plt.yscale('symlog', linthresh=0.1)
plt.xticks(range(len(labels)), labels)
plt.grid()
plt.legend(loc='best')
_ = plt.xticks(rotation=30)

## Benchmark of error estimation

Since for slice geometry the exact correction is known, we can benchmark
our method of its estimation.

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Convergence and true errors [V]')

plt.plot(true_error_L1,
         color=plt.plot(error_L1, label='L1')[0].get_color(),
         ls='--')

plt.plot(true_error_L2,
         color=plt.plot(error_L2, label='L2')[0].get_color(),
         ls='--')
plt.plot(true_error_Linf,
         color=plt.plot(error_Linf, label='L\u221e')[0].get_color(),
         ls='--')
plt.plot(true_error_bias,
         color=plt.plot(error_bias, label='bias')[0].get_color(),
         ls='--')
plt.yscale('symlog', linthresh=0.1)
plt.xticks(range(len(labels)), labels)
plt.grid()
plt.legend(loc='best')
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('Convergence and true errors [%]')

plt.plot(true_relative_error_L1 * 100,
         color=plt.plot(error_relative_L1 * 100, label='L1')[0].get_color(),
         ls='--')

plt.plot(true_relative_error_L2 * 100,
         color=plt.plot(error_relative_L2 * 100, label='L2')[0].get_color(),
         ls='--')
plt.plot(true_relative_error_Linf * 100,
         color=plt.plot(error_relative_Linf * 100, label='L\u221e')[0].get_color(),
         ls='--')
plt.plot(true_relative_error_bias * 100,
         color=plt.plot(error_relative_bias * 100, label='bias')[0].get_color(),
         ls='--')
plt.yscale('symlog', linthresh=1)
plt.xticks(range(len(labels)), labels)
plt.grid()
plt.legend(loc='best')
_ = plt.xticks(rotation=30)

In [ ]:
plt.figure(figsize=(16, 16))
plt.title('Errors vs estimates')
plt.xlabel('Error [V]')
plt.ylabel('Estimate [V]')


for _TRUE, _CONV in zip(gt_errors, diffs):
    plt.scatter(np.ravel(_TRUE), np.ravel(_CONV))

In [ ]:
plt.figure(figsize=(16, 16))
plt.title('Errors vs estimates')
plt.xlabel('Error [V]')
plt.ylabel('Estimate [V]')

plt.xscale('symlog')
plt.yscale('symlog')

for _TRUE, _CONV in zip(gt_errors, diffs):
    plt.scatter(np.ravel(_TRUE), np.ravel(_CONV))

In [ ]:
plt.figure(figsize=(16, 16))
plt.title('Errors vs estimates (diagonal removed)')
plt.xlabel('Error')
plt.ylabel('Estimate')

for _TRUE, _CONV in zip(gt_errors, diffs):
    plt.scatter(_TRUE[OFF_DIAGONAL_IDX], _CONV[OFF_DIAGONAL_IDX])

In [ ]:
plt.figure(figsize=(16, 16))
plt.title('Errors vs estimates (diagonal removed)')
plt.xlabel('Error [V]')
plt.ylabel('Estimate [V]')

plt.xscale('symlog')
plt.yscale('symlog')

plt.plot([-1e4, 1e4], [-1e4, 1e4], ls=':', color='k')
plt.plot([-1e4, 1e4], [1e4, -1e4], ls=':', color='k')
plt.axvline(0, ls=':', color='k')
plt.axhline(0, ls=':', color='k')
plt.axvline(1, ls=':', color='k')
plt.axhline(1, ls=':', color='k')
plt.axvline(-1, ls=':', color='k')
plt.axhline(-1, ls=':', color='k')

for _TRUE, _CONV in zip(gt_errors, diffs):
    plt.scatter(_TRUE[OFF_DIAGONAL_IDX], _CONV[OFF_DIAGONAL_IDX])

In [ ]:
for name, _TRUE, _CONV in zip(labels, gt_errors, diffs):
    plt.figure(figsize=(16, 16))
    plt.title(f'{name} (diagonal removed)')
    plt.xlabel('Error [V]')
    plt.ylabel('Estimate [V]')

    plt.xscale('symlog')
    plt.yscale('symlog')
    
    plt.xlim(-1e4, 1e4)
    plt.ylim(-1e4, 1e4)
    plt.plot([-1e4, 1e4], [-1e4, 1e4], ls=':', color='k')
    plt.plot([-1e4, 1e4], [1e4, -1e4], ls=':', color='k')
    plt.axvline(0, ls=':', color='k')
    plt.axhline(0, ls=':', color='k')
    plt.axvline(1, ls=':', color='k')
    plt.axhline(1, ls=':', color='k')
    plt.axvline(-1, ls=':', color='k')
    plt.axhline(-1, ls=':', color='k')
    plt.scatter(_TRUE[OFF_DIAGONAL_IDX], _CONV[OFF_DIAGONAL_IDX], alpha=0.5)

In [ ]:
plt.figure(figsize=(16, 16))
plt.title('Errors vs estimates (diagonal removed)')
plt.xlabel('Error [%]')
plt.ylabel('Estimate [%]')

plt.xscale('symlog')
plt.yscale('symlog')

plt.plot([-1e3, 1e3], [-1e3, 1e3], ls=':', color='k')
plt.plot([-1e3, 1e3], [1e3, -1e3], ls=':', color='k')
plt.axvline(0, ls=':', color='k')
plt.axhline(0, ls=':', color='k')
plt.axvline(1, ls=':', color='k')
plt.axhline(1, ls=':', color='k')
plt.axvline(-1, ls=':', color='k')
plt.axhline(-1, ls=':', color='k')

for _TRUE, _CONV in zip(gt_relative_errors, diffs_relative):
    plt.scatter(100 * _TRUE, 100 * _CONV[OFF_DIAGONAL_IDX])

In [ ]:
for name, _TRUE, _CONV in zip(labels, gt_relative_errors, diffs_relative):
    plt.figure(figsize=(16, 16))
    plt.title(f'{name} (diagonal removed)')
    plt.xlabel('Error [%]')
    plt.ylabel('Estimate [%]')

    plt.xscale('symlog')
    plt.yscale('symlog')
    
    plt.xlim(-1e3, 1e3)
    plt.ylim(-1e3, 1e3)
    plt.plot([-1e3, 1e3], [-1e3, 1e3], ls=':', color='k')
    plt.plot([-1e3, 1e3], [1e3, -1e3], ls=':', color='k')
    plt.axvline(0, ls=':', color='k')
    plt.axhline(0, ls=':', color='k')
    plt.axvline(1, ls=':', color='k')
    plt.axhline(1, ls=':', color='k')
    plt.axvline(-1, ls=':', color='k')
    plt.axhline(-1, ls=':', color='k')
    plt.scatter(100 * _TRUE, 100 * _CONV[OFF_DIAGONAL_IDX], alpha=0.5)